# Pacotes

In [2]:
# Padrões do Sistema e Ambientes
import os
import sys
import glob
import logging
import secrets
from functools import reduce

# Configuração de Ambientes Spark e Hadoop
print(os.environ.get("SPARK_HOME"))
print(os.environ.get("HADOOP_HOME"))
print(os.environ.get("JAVA_HOME"))
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

# Pacotes de Análise e Manipulação de Dados
import numpy as np
import pandas as pd
import chardet

# Spark
import pyspark
from pyspark import SparkContext, SQLContext, SparkConf, StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import (
    regexp_replace, when, length, to_date, upper, lower, col, split, explode,
    coalesce, concat_ws, concat, lit, broadcast, regexp_extract, month, year,
    expr, udf, row_number, isnan, count, monotonically_increasing_id, var_samp
)

# Pyspark Machine Learning e Ferramentas Estatísticas
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    StringIndexer, VectorAssembler, ChiSqSelector, VarianceThresholdSelector, 
    UnivariateFeatureSelector, RFormula, VectorSlicer, OneHotEncoder, StandardScaler
)
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.mllib.evaluation import MulticlassMetrics

# XGBoost para Spark
from xgboost.spark import SparkXGBClassifier

# Geopy para Geocodificação
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Hyperopt para Otimização de Hiperparâmetros
from hyperopt import STATUS_OK, Trials, fmin, hp, space_eval, tpe
from hyperopt.pyll import scope

# MLflow para Gerenciamento de Modelos
#import mlflow

# Visualização de Dados
import seaborn as sns
import matplotlib.pyplot as plt
import mpld3
import missingno as msno
import plotly.express as px
import plotly.offline as pyoff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

# Inicialização de Ferramentas
import findspark
findspark.init()
pyoff.init_notebook_mode(connected=True)

C:\Users\pedro\spark-3.5.0-bin-hadoop3
C:\Users\pedro\hadoop3.0
C:\Program Files\Java\jdk1.8.0_202


# Spark Session

In [3]:
spark = (SparkSession.builder 
    .master("local[*]") 
    .appName("Spark Optimization")   
    .config("spark.driver.cores", "3")   # Alocando 3 núcleos para o driver
    .config("spark.driver.memory", "12g")  # 12 GB de RAM para o driver
    .config("spark.executor.instances", "5")   # Configurando para 5 executores
    .config("spark.executor.cores", "2")   # Cada executor terá 2 núcleos
    .config("spark.executor.memory", "6g")   # Cada executor terá 6 GB de RAM
    .config("spark.executor.memoryOverhead", "2g")   # Overhead adicional para evitar spill para disco
    .config("spark.memory.fraction", "0.6")  
    .config("spark.memory.storageFraction", "0.5")   
    .config("spark.memory.offHeap.enabled", "true")   
    .config("spark.memory.offHeap.size", "4g")   # Memória off-heap adicional para operações fora do heap JVM
    .config("spark.driver.maxResultSize", "4g")   
    .config("spark.sql.autoBroadcastJoinThreshold", "400m")  # Broadcast join otimizado para joins pequenos
    .config("spark.default.parallelism", "32")   # Paralelismo adequado ao tamanho do dataset
    .config("spark.sql.shuffle.partitions", "32")   # Número de partições para operações de shuffle
    .config("spark.sql.repl.eagerEval.enabled", True)   
    .config("spark.sql.adaptive.enabled", True) 
    .config("spark.sql.cbo.enabled", True) 
    .config("spark.sql.repl.eagerEval.maxNumRows", 10)  
    .config("spark.shuffle.compress", "true")   
    .config("spark.storage.level", "MEMORY_AND_DISK")   
    .config("spark.rdd.compress", "true")   
    .config("spark.serializer", "org.apache.spark.serializer.JavaSerializer")
    .getOrCreate())

In [4]:
#spark.stop()

In [6]:
spark

In [30]:
def feature_importance_logistic_reg(df, categorical_cols, numeric_features, response_col):
    # Indexação das colunas categóricas
    indexers = [
        StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip")
        for col in categorical_cols
    ]

    # Indexação da coluna de resposta
    indexers.append(StringIndexer(inputCol=response_col, outputCol=response_col + "_index", handleInvalid="skip"))

    # Assembler para criar o vetor de features
    input_cols = [col + "_index" for col in categorical_cols] + numeric_features
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

    # Pipeline para processar indexação e assembler
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Configurando a Regressão Logística com ElasticNet
    lr = LogisticRegression(featuresCol="features", labelCol=response_col + "_index", elasticNetParam=0.5)
    lr_model = lr.fit(df_transformed)

    # Obtendo os coeficientes das features
    coefficients = lr_model.coefficients.toArray()

    # Vinculando coeficientes com os nomes das colunas
    important_features = sorted(zip(input_cols, coefficients), key=lambda x: abs(x[1]), reverse=True)

    # Definindo um threshold para selecionar as colunas mais importantes
    threshold = 0.05  # Ajuste conforme necessário
    significant_columns = [name for name, coef in important_features if abs(coef) > threshold]

    print("Colunas significativas com base na Regressão Logística ElasticNet:")
    print(significant_columns)
    
    return significant_columns

# Chamando a função para regressão logística


def feature_importance_linear_reg(df, categorical_cols, numeric_features, response_col):
    # Indexação das colunas categóricas
    indexers = [
        StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip")
        for col in categorical_cols
    ]

    # Assembler para criar o vetor de features
    input_cols = [col + "_index" for col in categorical_cols] + numeric_features
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

    # Pipeline para processar indexação e assembler
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Configurando a Regressão Linear com ElasticNet (regularização combinando L1 e L2)
    lr = LinearRegression(featuresCol="features", labelCol=response_col, elasticNetParam=0.5)
    lr_model = lr.fit(df_transformed)

    # Obtendo os coeficientes das features
    coefficients = lr_model.coefficients.toArray()

    # Vinculando coeficientes com os nomes das colunas
    important_features = sorted(zip(input_cols, coefficients), key=lambda x: abs(x[1]), reverse=True)

    # Definindo um threshold para selecionar as colunas mais importantes
    threshold = 0.05  # Ajuste conforme necessário
    significant_columns = [name for name, coef in important_features if abs(coef) > threshold]

    print("Colunas significativas com base na Regressão Linear ElasticNet:")
    print(significant_columns)
    
    return significant_columns

def feature_importance_rf_reg(df, categorical_cols, numeric_features, response_col):
    # Remover a variável dependente das listas de variáveis preditoras
    if response_col in categorical_cols:
        categorical_cols.remove(response_col)
    if response_col in numeric_features:
        numeric_features.remove(response_col)

    # Indexação das colunas categóricas
    indexers = [
        StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip")
        for col in categorical_cols
    ]

    # Assembler para criar o vetor de features
    input_cols = [col + "_index" for col in categorical_cols] + numeric_features
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

    # Pipeline para processar indexação e assembler
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Treinando um RandomForestRegressor para avaliar a importância das features com maxBins ajustado
    rf = RandomForestRegressor(featuresCol="features", labelCol=response_col, maxBins=5500)
    rf_model = rf.fit(df_transformed)

    # Obtendo as importâncias das features
    importances = rf_model.featureImportances

    # Vinculando importâncias com os nomes das colunas
    important_features = sorted(zip(input_cols, importances), key=lambda x: abs(x[1]), reverse=True)

    # Definindo um threshold para selecionar as colunas mais importantes
    threshold = 0.05  # Ajuste conforme necessário
    significant_columns = [name for name, importance in important_features if importance > threshold]

    print("Colunas significativas com base na importância do RandomForestRegressor:")
    print(significant_columns)
    
    return significant_columns

def feature_importance_rf_class(df, categorical_cols, numeric_features, response_col):
    # Limpeza e preparação das colunas
    df = clean_up_columns(df, response_col)
    
    # Definir as colunas categóricas e numéricas se necessário
    categorical_cols = [t[0] for t in df.dtypes if t[1] == 'string']
    numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]

    # Garantir que a variável resposta não esteja nas listas
    if response_col in categorical_cols:
        categorical_cols.remove(response_col)
    if response_col in numeric_features:
        numeric_features.remove(response_col)

    # Indexação das colunas categóricas
    indexers = [
        StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip")
        for col in categorical_cols
    ]

    # Indexação da coluna de resposta
    indexers.append(StringIndexer(inputCol=response_col, outputCol=response_col + "_index", handleInvalid="skip"))

    # Assembler para criar o vetor de features
    input_cols = [col + "_index" for col in categorical_cols] + numeric_features
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

    # Pipeline para processar indexação e assembler
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Treinando um RandomForest para avaliar a importância das features
    rf = RandomForestClassifier(featuresCol="features", labelCol=response_col + "_index")
    rf_model = rf.fit(df_transformed)

    # Obtendo as importâncias das features
    importances = rf_model.featureImportances

    # Vinculando importâncias com os nomes das colunas
    important_features = sorted(zip(input_cols, importances), key=lambda x: x[1], reverse=True)

    # Definindo um threshold para selecionar as colunas mais importantes
    threshold = 0.05  # Ajuste conforme necessário
    significant_columns = [name for name, importance in important_features if importance > threshold]

    print("Colunas significativas com base na importância do RandomForest:")
    print(significant_columns)
    
    return significant_columns

def univariate_feature_selector(df, categorical_cols, numeric_features, response_col, selectionThreshold=5):
    # Limpeza e preparação das colunas
    df = clean_up_columns(df, response_col)

    # Indexação de colunas categóricas
    indexers = [StringIndexer(inputCol=c, outputCol=c + "_index", handleInvalid="skip") for c in categorical_cols]

    # Indexação da coluna de resposta
    indexer_response = StringIndexer(inputCol=response_col, outputCol="label", handleInvalid="skip")

    # Assembler para combinar colunas numéricas e categóricas indexadas em um vetor de features
    input_cols = [c + "_index" for c in categorical_cols] + numeric_features
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

    # Configuração do UnivariateFeatureSelector
    selector = UnivariateFeatureSelector(
        featuresCol="features",
        outputCol="selectedFeatures",
        labelCol="label",
        selectionMode="numTopFeatures"
    ).setFeatureType("continuous").setLabelType("categorical").setSelectionThreshold(selectionThreshold)

    # Pipeline para realizar a transformação
    pipeline = Pipeline(stages=indexers + [indexer_response, assembler, selector])
    model = pipeline.fit(df)
    df_transformed = model.transform(df)

    # Obter os índices das features selecionadas
    selected_indices = model.stages[-1].selectedFeatures  # Último estágio é o UnivariateFeatureSelector

    # Mapear os índices selecionados de volta para os nomes das colunas
    selected_feature_names = [input_cols[index] for index in selected_indices]

    # Imprimir os nomes das colunas das features selecionadas
    print("Colunas selecionadas:", selected_feature_names)

    return selected_feature_names

def missing_values_table_spark(df):
    """
    Cria uma tabela resumindo a quantidade e a porcentagem de valores ausentes em cada coluna do DataFrame PySpark.

    Args:
    df (spark.DataFrame): DataFrame para análise de valores ausentes.

    Returns:
    spark.DataFrame: Uma tabela com o número e a porcentagem de valores ausentes por coluna.
    """
    # Calcula o número total de linhas no DataFrame
    total_rows = df.count()

    # Calcula o número total de valores ausentes por coluna
    missing_count = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

    # Calcula a porcentagem de valores ausentes por coluna
    missing_percent = df.select([(count(when(col(c).isNull(), c)) / total_rows * 100).alias(c) for c in df.columns])

    # Preparando para juntar contagens e percentagens
    missing_count = missing_count.withColumnRenamed(missing_count.columns[0], 'Missing Values')
    missing_percent = missing_percent.withColumnRenamed(missing_percent.columns[0], '% of Total Values')

    # Junta as contagens e as percentagens em um DataFrame
    missing_table = missing_count.join(missing_percent)

    # Ordena as colunas com valores ausentes por porcentagem de forma decrescente
    missing_table = missing_table.orderBy(col('% of Total Values').desc())

    # Imprime um resumo das colunas com valores ausentes
    print("Your selected dataframe has " + str(len(df.columns)) + " columns.\n"      
          "There are " + str(missing_table.count()) + " columns that have missing values.")

    return missing_table



def r_formula_with_feature_importance(df, categorical_cols, numeric_features, response_col):
    # Definindo a fórmula
    predictor_cols = categorical_cols + numeric_features
    formula_expression = f"{response_col} ~ " + " + ".join(predictor_cols)

    # Configuração do RFormula
    formula = RFormula(
        formula=formula_expression,
        featuresCol="features",
        labelCol="label"
    )

    # Pipeline para aplicar a RFormula
    pipeline = Pipeline(stages=[formula])
    df_transformed = pipeline.fit(df).transform(df)

    # Dividindo os dados em treino e teste (opcional)
    train_df, test_df = df_transformed.randomSplit([0.75, 0.25], seed=1234)

    # Usando RandomForest para avaliar a importância das features
    rf = RandomForestRegressor(featuresCol="features", labelCol="label")
    rf_model = rf.fit(train_df)

    # Obtendo as importâncias das features
    importances = rf_model.featureImportances

    # Vinculando importâncias com os nomes das colunas preditivas
    feature_list = categorical_cols + numeric_features
    important_features = sorted(zip(feature_list, importances), key=lambda x: x[1], reverse=True)

    # Exibindo as colunas mais significativas
    print("Colunas significativas com base na importância do RandomForest:")
    for feature, importance in important_features:
        print(f"Feature: {feature}, Importance: {importance}")

    # Avaliar o modelo no conjunto de teste (opcional)
    predictions = rf_model.transform(test_df)
    evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    print(f"Root Mean Squared Error (RMSE) no conjunto de teste: {rmse}")
    
    return important_features

def vector_slicer_function(df, categorical_cols, numeric_features, response_col):
    # Limpeza e preparação das colunas
    df = clean_up_columns(df, response_col)

    # Indexação das colunas categóricas
    indexers = [StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip") for col in categorical_cols]

    # Assembler para combinar as colunas indexadas e numéricas em um vetor de features
    input_cols = [col + "_index" for col in categorical_cols] + numeric_features
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

    # Pipeline para processar a indexação e assembler
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Slicing das features
    indices_to_slice = [0, 1, 3, 4, 5]  # Ajuste conforme necessário
    vs = VectorSlicer(inputCol="features", outputCol="selectedFeatures", indices=indices_to_slice)
    df_sliced = vs.transform(df_transformed)

    # Mapeando os índices para os nomes das colunas
    selected_feature_names = [input_cols[index] for index in indices_to_slice]

    # Exibindo as features selecionadas e seus nomes
    df_sliced.select("selectedFeatures").show(truncate=False)
    print("Nomes das colunas selecionadas:", selected_feature_names)

    return selected_feature_names


def variance_threshold_selector(df, numeric_features, varianceThreshold=0.5):
    # Assembler para criar o vetor de features numéricas
    assembler = VectorAssembler(
        inputCols=numeric_features,
        outputCol="features")

    # Aplicar o assembler ao DataFrame
    df_assembled = assembler.transform(df)

    # Configurando o VarianceThresholdSelector
    selector = VarianceThresholdSelector(
        featuresCol="features",
        outputCol="selectedFeatures",
        varianceThreshold=varianceThreshold  # Definir um limiar de variância apropriado
    )

    # Aplicando o selector ao DataFrame
    model = selector.fit(df_assembled)
    df_selected = model.transform(df_assembled)

    # Obter os índices das features selecionadas
    selected_indices = model.selectedFeatures

    # Criar uma lista de nomes de colunas mapeando de volta usando os índices selecionados
    selected_feature_names = [numeric_features[index] for index in selected_indices]

    print("Colunas selecionadas com variância acima do limiar:", selected_feature_names)

    return selected_feature_names

def chi_sq_selector(df, categorical_cols, response_col, numTopFeatures=4):
    # Indexação de colunas categóricas
    indexers = [StringIndexer(inputCol=c, outputCol=c + "_index") for c in categorical_cols] + \
               [StringIndexer(inputCol=response_col, outputCol="label")]

    # Assembler para combinar as colunas categóricas indexadas em um vetor de features
    assembler = VectorAssembler(
        inputCols=[c + "_index" for c in categorical_cols],
        outputCol="features")

    # Configuração do ChiSqSelector
    selector = ChiSqSelector(numTopFeatures=numTopFeatures, featuresCol="features",
                             outputCol="selectedFeatures", labelCol="label")

    # Pipeline para processar tudo
    pipeline = Pipeline(stages=indexers + [assembler, selector])
    model = pipeline.fit(df)
    df_transformed = model.transform(df)

    # Obter o modelo do selector para acessar os índices das features selecionadas
    selected_features_model = model.stages[-1]
    selected_indices = selected_features_model.selectedFeatures

    # Criar uma lista de nomes de colunas mapeando de volta usando os índices selecionados
    selected_feature_names = [categorical_cols[index] for index in selected_indices]

    print("Colunas selecionadas pelo teste Chi-quadrado:")
    print(selected_feature_names)

    return selected_feature_names

def clean_up_columns(df, response_col):
    if "label" in df.columns:
        df = df.drop("label")
    if f"{response_col}_index" in df.columns:
        df = df.drop(f"{response_col}_index")
    if f"old_{response_col}_index" in df.columns:
        df = df.drop(f"old_{response_col}_index")
    return df

def univariate_feature_selector_var_num(df, categorical_cols, numeric_features, response_col, selectionThreshold=5):
    # Limpeza de colunas, se necessário
    df = clean_up_columns(df, response_col)

    # Garantir que a variável dependente não esteja nas listas de variáveis preditivas
    if response_col in categorical_cols:
        categorical_cols.remove(response_col)
    if response_col in numeric_features:
        numeric_features.remove(response_col)

    # Indexação de colunas categóricas
    indexers = [StringIndexer(inputCol=c, outputCol=c + "_index", handleInvalid="keep") for c in categorical_cols]

    # Assembler para combinar as colunas categóricas indexadas e as numéricas em um vetor de features
    assembler = VectorAssembler(
        inputCols=[c + "_index" for c in categorical_cols] + numeric_features,
        outputCol="features"
    )

    # Definir o tipo de seleção baseado no tipo da variável dependente
    # Se a variável resposta for categórica, use "categorical", caso contrário "continuous"
    if df.schema[response_col].dataType.simpleString() in ['int', 'double', 'float', 'long']:
        label_type = "continuous"
    else:
        label_type = "categorical"

    # Configurar o UnivariateFeatureSelector
    selector = UnivariateFeatureSelector(
        featuresCol="features",
        outputCol="selectedFeatures",
        labelCol=response_col,  # Usar a coluna de resposta diretamente
        selectionMode="numTopFeatures"
    )

    selector.setFeatureType("continuous").setLabelType(label_type).setSelectionThreshold(selectionThreshold)

    # Pipeline para transformar os dados
    pipeline = Pipeline(stages=indexers + [assembler, selector])
    model = pipeline.fit(df)
    df_transformed = model.transform(df)

    # Obter os índices das features selecionadas
    selected_indices = model.stages[-1].selectedFeatures  # O último estágio é o UnivariateFeatureSelector

    # Lista de todas as colunas que foram usadas para criar o vetor de features
    all_features = [col + "_index" for col in categorical_cols] + numeric_features

    # Mapear os índices selecionados de volta para os nomes das colunas
    selected_feature_names = [all_features[index] for index in selected_indices]

    # Imprimir os nomes das colunas das features selecionadas
    print("Colunas selecionadas:", selected_feature_names)

    return selected_feature_names


def feature_importance_linear_reg(df, categorical_cols, numeric_features, response_col):
    # Remover a variável dependente das listas de variáveis preditoras
    if response_col in categorical_cols:
        categorical_cols.remove(response_col)
    if response_col in numeric_features:
        numeric_features.remove(response_col)

    # Indexação das colunas categóricas
    indexers = [
        StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip")
        for col in categorical_cols
    ]

    # Assembler para criar o vetor de features
    input_cols = [col + "_index" for col in categorical_cols] + numeric_features
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

    # Pipeline para processar indexação e assembler
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Configurando a Regressão Linear com ElasticNet (regularização combinando L1 e L2)
    lr = LinearRegression(featuresCol="features", labelCol=response_col, elasticNetParam=0.5)
    lr_model = lr.fit(df_transformed)

    # Obtendo os coeficientes das features
    coefficients = lr_model.coefficients.toArray()

    # Vinculando coeficientes com os nomes das colunas
    important_features = sorted(zip(input_cols, coefficients), key=lambda x: abs(x[1]), reverse=True)

    # Definindo um threshold para selecionar as colunas mais importantes
    threshold = 0.05  # Ajuste conforme necessário
    significant_columns = [name for name, coef in important_features if abs(coef) > threshold]

    print("Colunas significativas com base na Regressão Linear ElasticNet:")
    print(significant_columns)
    
    return significant_columns

def pearson_correlation(df, numeric_features):
    corr_matrix = []
    for x in numeric_features:
        temp = []
        for y in numeric_features:
            temp.append(df.stat.corr(x, y, method='pearson'))
        corr_matrix.append(temp)
    corr_df = pd.DataFrame(corr_matrix, index=numeric_features, columns=numeric_features)
    
    ## corr_df.to_csv("pearson_correlation.csv", index=False)
    return corr_df

# Dataset

In [8]:
df = spark.read.parquet('C:/Users/pedro/Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_final_filtrado_s_missing/').cache()

In [9]:
df = df.drop(df.NM_MATRIZ_FILIAL,df.NAT_JURICA , df.NM_QUALIFICACAO,df.CEP)

## Missing

In [10]:
# Especificar as colunas que você deseja transformar
columns_to_cast = [
    "LON",
    "LAT"
]

# Substituir ',' por '.' nas colunas selecionadas
for column in columns_to_cast:
    df = df.withColumn(column, F.regexp_replace(F.col(column), ',', '.'))

# Aplicar a transformação de tipo para FLOAT
for column in columns_to_cast:
    df = df.withColumn(column, F.col(column).cast("float"))

In [11]:
missing_values_table_spark(df)

Your selected dataframe has 24 columns.
There are 1 columns that have missing values.


Missing Values,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,SEXO_PROB,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010,% of Total Values,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,SEXO_PROB,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feature Selection

In [12]:
def clean_up_columns(df, response_col):
    if "label" in df.columns:
        df = df.drop("label")
    if f"{response_col}_index" in df.columns:
        df = df.drop(f"{response_col}_index")
    if f"old_{response_col}_index" in df.columns:
        df = df.drop(f"old_{response_col}_index")
    return df


def univariate_feature_selector(df, categorical_cols, numeric_features, response_col, selectionThreshold=5):
    df = clean_up_columns(df, response_col)
    
    # Lista de colunas numéricas
    numeric_features = numeric_features

    # Identificar as colunas categóricas
    categorical_cols = categorical_cols

    # Supondo que uma das colunas numéricas seja a resposta e precisamos transformá-la
    response_col = 'STATUS'  # Variável a ser predita

    # Indexação de colunas categóricas e coluna de resposta
    indexers = [StringIndexer(inputCol=c, outputCol=c+"_index", handleInvalid="keep") for c in categorical_cols]
    indexer_response = StringIndexer(inputCol="STATUS", outputCol="label", handleInvalid="keep")

    # Assembler para combinar colunas numéricas e categóricas indexadas em um vetor de features
    assembler = VectorAssembler(
        inputCols=[c+"_index" for c in categorical_cols] + numeric_features,
        outputCol="features")

    selector = UnivariateFeatureSelector(
        featuresCol="features",
        outputCol="selectedFeatures",
        labelCol="label",  # Usar "label", pois é o nome configurado pelo indexer_response
        selectionMode="numTopFeatures"  # ou use outro modo conforme necessário
    )

    selector.setFeatureType("continuous").setLabelType("categorical").setSelectionThreshold(selectionThreshold)  # ajuste conforme necessário

    # Adicionando a transformação da label ao pipeline
    pipeline = Pipeline(stages=indexers + [indexer_response, assembler, selector])
    model = pipeline.fit(df)
    df_transformed = model.transform(df)

    # Obter os índices das features selecionadas
    selected_indices = model.stages[-1].selectedFeatures  # O último estágio é o UnivariateFeatureSelector

    # Lista de todas as colunas que foram usadas para criar o vetor de features
    all_features = [col + "_index" for col in categorical_cols] + numeric_features

    # Mapear os índices selecionados de volta para os nomes das colunas
    selected_feature_names = [all_features[index] for index in selected_indices]

    # Imprimir os nomes das colunas das features selecionadas
    print("Colunas selecionadas:", selected_feature_names)

    return selected_feature_names




# Criação de um index único

In [21]:
df = df.withColumn("index", monotonically_increasing_id())

# Execução

In [13]:
df.printSchema()

root
 |-- CNPJ: string (nullable = true)
 |-- NOME_EMPRESA: string (nullable = true)
 |-- NOME_FANTASIA: string (nullable = true)
 |-- CAP_SOCIAL: float (nullable = true)
 |-- SEXO_PROB: string (nullable = true)
 |-- CNAE: string (nullable = true)
 |-- ano_cadastro: integer (nullable = true)
 |-- mes_cadastro: integer (nullable = true)
 |-- TIPO_LOUGRADOURO: string (nullable = true)
 |-- CEP: integer (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- PROVEDOR: string (nullable = true)
 |-- LON: float (nullable = true)
 |-- LAT: float (nullable = true)
 |-- DDD1: string (nullable = true)
 |-- TIPO_TELEFONE_FINAL: string (nullable = true)
 |-- OPERADORA: string (nullable = true)
 |-- renda_per_capita: float (nullable = true)
 |-- PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000: float (nullable = true)
 |-- PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100: float (nullable = true)
 |-- ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO: string 

## Correlação de Pearson

In [16]:
numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]
corr_df = pearson_correlation(df, numeric_features)

In [17]:
corr_df

,CAP_SOCIAL,ano_cadastro,mes_cadastro,LON,LAT,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,IDHM_2010
CAP_SOCIAL,1.000000e+00,0.001415,0.000465,0.000315,6.401356e-07,-0.000120,-0.000101,-0.000491,-0.000922
ano_cadastro,1.415190e-03,1.000000,-0.050614,-0.025593,-3.803530e-02,0.014278,-0.012613,-0.005944,0.046667
mes_cadastro,4.653095e-04,-0.050614,1.000000,0.014502,2.530920e-03,0.001000,0.014670,0.022993,0.008726
LON,3.154417e-04,-0.025593,0.014502,1.000000,3.535273e-01,-0.097565,0.003039,-0.119251,-0.196605
LAT,6.401356e-07,-0.038035,0.002531,0.353527,1.000000e+00,-0.205726,-0.181833,-0.247140,-0.470019
renda_per_capita,-1.198429e-04,0.014278,0.001000,-0.097565,-2.057262e-01,1.000000,0.217996,0.168798,0.404073
PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,-1.005979e-04,-0.012613,0.014670,0.003039,-1.818333e-01,0.217996,1.000000,0.330609,0.378416
PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,-4.914107e-04,-0.005944,0.022993,-0.119251,-2.471400e-01,0.168798,0.330609,1.000000,0.353570
IDHM_2010,-9.217829e-04,0.046667,0.008726,-0.196605,-4.700186e-01,0.404073,0.378416,0.353570,1.000000


## Variance Threshold Selector

In [18]:
# Variance Threshold Selector
response_col = 'CAP_SOCIAL'
numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]
selected_features_variance = variance_threshold_selector(df, numeric_features)
selected_features_variance

Colunas selecionadas com variância acima do limiar: ['CAP_SOCIAL', 'ano_cadastro', 'mes_cadastro', 'LON', 'LAT', 'renda_per_capita', 'PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000', 'PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100']


['CAP_SOCIAL',
 'ano_cadastro',
 'mes_cadastro',
 'LON',
 'LAT',
 'renda_per_capita',
 'PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000',
 'PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100']

## RFormula

In [19]:
# RFormula
response_col = 'CAP_SOCIAL'
categorical_cols = ["SEXO_PROB", "CNAE", "TIPO_LOUGRADOURO", 
                    "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF",
                    "ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO", "ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO"]
r_formula_df = r_formula_with_feature_importance(df, categorical_cols, numeric_features, response_col)
r_formula_df

Colunas significativas com base na importância do RandomForest:
Feature: TIPO_LOUGRADOURO, Importance: 0.03138327609303246
Feature: UF, Importance: 0.024775483506614855
Feature: SEXO_PROB, Importance: 0.01660597854927476
Feature: ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO, Importance: 2.5326721948599455e-05
Feature: CNAE, Importance: 4.2959503610569925e-06
Feature: mes_cadastro, Importance: 8.225645851901859e-11
Feature: PROVEDOR, Importance: 4.859919195163692e-13
Feature: DDD1, Importance: 0.0
Feature: TIPO_TELEFONE_FINAL, Importance: 0.0
Feature: OPERADORA, Importance: 0.0
Feature: MUNICIPIO, Importance: 0.0
Feature: ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO, Importance: 0.0
Feature: CAP_SOCIAL, Importance: 0.0
Feature: ano_cadastro, Importance: 0.0
Feature: LON, Importance: 0.0
Feature: LAT, Importance: 0.0
Feature: renda_per_capita, Importance: 0.0
Feature: PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000, Importance: 0.0
Feature: PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_COR

[('TIPO_LOUGRADOURO', 0.03138327609303246),
 ('UF', 0.024775483506614855),
 ('SEXO_PROB', 0.01660597854927476),
 ('ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO', 2.5326721948599455e-05),
 ('CNAE', 4.2959503610569925e-06),
 ('mes_cadastro', 8.225645851901859e-11),
 ('PROVEDOR', 4.859919195163692e-13),
 ('DDD1', 0.0),
 ('TIPO_TELEFONE_FINAL', 0.0),
 ('OPERADORA', 0.0),
 ('MUNICIPIO', 0.0),
 ('ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO', 0.0),
 ('CAP_SOCIAL', 0.0),
 ('ano_cadastro', 0.0),
 ('LON', 0.0),
 ('LAT', 0.0),
 ('renda_per_capita', 0.0),
 ('PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000', 0.0),
 ('PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100', 0.0),
 ('IDHM_2010', 0.0)]

## VectorSlicer

In [20]:
# VectorSlicer
response_col = 'CAP_SOCIAL'
categorical_cols = ["SEXO_PROB", "CNAE", "TIPO_LOUGRADOURO", 
                    "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF",
                    "ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO", "ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO"]
numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]

vector_resp = vector_slicer_function(df, categorical_cols, numeric_features, response_col)

+------------------------+
|selectedFeatures        |
+------------------------+
|[0.0,18.0,1.0,14.0,0.0] |
|[0.0,42.0,1.0,8.0,0.0]  |
|[0.0,20.0,1.0,5.0,0.0]  |
|[0.0,56.0,17.0,6.0,0.0] |
|[0.0,61.0,1.0,21.0,0.0] |
|[0.0,20.0,1.0,51.0,0.0] |
|[0.0,168.0,1.0,1.0,0.0] |
|[1.0,102.0,1.0,21.0,0.0]|
|[0.0,5.0,0.0,2.0,0.0]   |
|[0.0,216.0,1.0,20.0,0.0]|
|[1.0,88.0,3.0,14.0,0.0] |
|[0.0,21.0,5.0,6.0,0.0]  |
|[0.0,6.0,0.0,45.0,0.0]  |
|(5,[1,2],[21.0,6.0])    |
|[1.0,4.0,0.0,1.0,0.0]   |
|[1.0,0.0,4.0,22.0,0.0]  |
|[0.0,40.0,3.0,14.0,0.0] |
|[0.0,82.0,4.0,36.0,0.0] |
|[0.0,115.0,4.0,24.0,0.0]|
|[1.0,295.0,3.0,21.0,0.0]|
+------------------------+
only showing top 20 rows

Nomes das colunas selecionadas: ['SEXO_PROB_index', 'CNAE_index', 'PROVEDOR_index', 'DDD1_index', 'TIPO_TELEFONE_FINAL_index']


## Random Forest

In [27]:
response_col = 'CAP_SOCIAL'
categorical_cols = ["SEXO_PROB", "CNAE", "TIPO_LOUGRADOURO", 
                    "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF",
                    "ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO", "ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO"]
numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]
significant_columns_rf_reg = feature_importance_rf_reg(df, categorical_cols, numeric_features, response_col)

Colunas significativas com base na importância do RandomForestRegressor:
['CNAE_index', 'PROVEDOR_index', 'ano_cadastro', 'MUNICIPIO_index', 'TIPO_TELEFONE_FINAL_index']


In [29]:
significant_columns_rf_reg

['CNAE_index',
 'PROVEDOR_index',
 'ano_cadastro',
 'MUNICIPIO_index',
 'TIPO_TELEFONE_FINAL_index']

## Univariate

In [21]:
response_col = 'CAP_SOCIAL'
categorical_cols = ["SEXO_PROB", "CNAE", "TIPO_LOUGRADOURO", 
                    "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF",
                    "ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO", "ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO"]
numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]

selected_features = univariate_feature_selector_var_num(df, categorical_cols, numeric_features, response_col, selectionThreshold=10)

Colunas selecionadas: ['CNAE_index', 'TIPO_TELEFONE_FINAL_index', 'OPERADORA_index', 'DDD1_index', 'ano_cadastro', 'IDHM_2010', 'ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO_index', 'UF_index', 'ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO_index', 'PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100']


## Regressão 

In [24]:
def feature_importance_linear_reg(df, categorical_cols, numeric_features, response_col):
    # Remover a variável dependente das listas de variáveis preditoras
    if response_col in categorical_cols:
        categorical_cols.remove(response_col)
    if response_col in numeric_features:
        numeric_features.remove(response_col)

    # Indexação das colunas categóricas
    indexers = [
        StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip")
        for col in categorical_cols
    ]

    # Assembler para criar o vetor de features
    input_cols = [col + "_index" for col in categorical_cols] + numeric_features
    assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

    # Pipeline para processar indexação e assembler
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Configurando a Regressão Linear com ElasticNet (regularização combinando L1 e L2)
    lr = LinearRegression(featuresCol="features", labelCol=response_col, elasticNetParam=0.5)
    lr_model = lr.fit(df_transformed)

    # Obtendo os coeficientes das features
    coefficients = lr_model.coefficients.toArray()

    # Vinculando coeficientes com os nomes das colunas
    important_features = sorted(zip(input_cols, coefficients), key=lambda x: abs(x[1]), reverse=True)

    # Definindo um threshold para selecionar as colunas mais importantes
    threshold = 0.05  # Ajuste conforme necessário
    significant_columns = [name for name, coef in important_features if abs(coef) > threshold]

    print("Colunas significativas com base na Regressão Linear ElasticNet:")
    print(significant_columns)
    
    return significant_columns

In [25]:
# Chamando a função para regressão linear
response_col = 'CAP_SOCIAL'
categorical_cols = ["SEXO_PROB", "CNAE", "TIPO_LOUGRADOURO", 
                    "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF",
                    "ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO", "ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO"]
numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]
significant_columns_linear = feature_importance_linear_reg(df, categorical_cols, numeric_features, response_col)

Colunas significativas com base na Regressão Linear ElasticNet:
['IDHM_2010', 'TIPO_TELEFONE_FINAL_index', 'SEXO_PROB_index', 'ano_cadastro', 'ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO_index', 'ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO_index', 'UF_index', 'mes_cadastro', 'OPERADORA_index', 'DDD1_index', 'LON', 'CNAE_index', 'TIPO_LOUGRADOURO_index', 'LAT', 'PROVEDOR_index', 'MUNICIPIO_index', 'renda_per_capita']
